In [1]:
from itertools import product
from functools import partial

import pandas as pd

from superfv import (
    EulerSolver,
    OutputLoader,
)
from superfv.initial_conditions import square

In [2]:
N = 64
T = 1.0
gamma = 1.4

base = "../out/square2d/"  # "/scratch/gpfs/jp7427/out/square2d/"
overwrite = []

configs = {
    "ZS3": dict(
        GL=True,
        riemann_solver="hllc",
        flux_recipe=2,
        lazy_primitives=True,
        p=3,
        ZS=True,
        include_corners=True,
        adaptive_dt=False,
        PAD={"rho": (0, None), "P": (0, None)},
        SED=True,
    ),
    # "ZS3-T": dict(
    #     GL=False,
    #     riemann_solver="hllc",
    #     flux_recipe=2,
    #     lazy_primitives=True,
    #     p=3,
    #     ZS=True,
    #     include_corners=True,
    #     adaptive_dt=False,
    #     PAD={"rho": (0, None), "P": (0, None)},
    #     SED=True,
    # ),
    # "MM3": dict(
    #     riemann_solver="hllc",
    #     flux_recipe=2,
    #     lazy_primitives=True,
    #     p=3,
    #     MOOD=True,
    #     cascade="muscl",
    #     MUSCL_limiter="moncen",
    #     max_MOOD_iters=1,
    #     limiting_vars="actives",
    #     NAD=True,
    #     include_corners=True,
    #     NAD_rtol=1e-2,
    #     NAD_atol=1e-2,
    #     PAD={"rho": (0, None), "P": (0, None)},
    #     SED=True,
    # ),
}

sims = {}
data = []
for (name, config), rho_min in product(configs.items(), [1e-1]):
    path = base + name + f"/rho_min_{rho_min:.0e}/"
    if overwrite != "all":
        if name not in overwrite:
            try:
                sim = OutputLoader(path)
                sims[name] = sim
                with open(path + "status.txt", "r") as f:
                    status = f.read().strip()
                data.append(
                    dict(
                        name=name,
                        rho_min=rho_min,
                        passed={"passed": True, "failed": False}.get(status, None),
                    )
                )
                continue
            except FileNotFoundError:
                pass

    print(f"Running {name}...")
    sim = EulerSolver(
        ic=partial(square, vx=2, vy=1, bounds=(rho_min, 1.0), P=1e-5),
        gamma=gamma,
        nx=N,
        ny=N,
        cupy=True,
        **config,
    )

    try:
        if config.get("MUSCL", False):
            sim.musclhancock(
                T, allow_overshoot=True, path=path, overwrite=True, snapshot_mode="none"
            )
        else:
            sim.run(
                T,
                q_max=2,
                allow_overshoot=True,
                path=path,
                overwrite=True,
                snapshot_mode="none",
            )
        sims[name] = sim
        with open(path + "status.txt", "w") as f:
            f.write("passed")
        data.append(dict(name=name, rho_min=rho_min, passed=True))
    except Exception as e:
        print(f"Failed: {e}")
        with open(path + "status.txt", "w") as f:
            f.write("failed")
        data.append(dict(name=name, rho_min=rho_min, passed=False))
        continue
df = pd.DataFrame(data)

Running ZS3...
Overwriting existing output directory "../out/square2d/ZS3/rho_min_1e-01".
                                                                                                    

/Users/jonathan/Desktop/superfv/superfv/finite_volume_solver.py:353: UserWarning: CuPy is not available. Using NumPy instead.
  warnings.warn("CuPy is not available. Using NumPy instead.")


Step #200 @ t=8.27e-01 | dt=4.13e-03 | min(rho)=9.82e-02, min(P)=1.00e-05                           

KeyboardInterrupt: 

In [ ]:
df

,name,rho_min,passed
0,ZS3,1.000000e-06,True
1,ZS3,1.000000e-07,True
2,ZS3,1.000000e-08,True
3,ZS3-T,1.000000e-06,False
4,ZS3-T,1.000000e-07,False
5,ZS3-T,1.000000e-08,False
6,MM3,1.000000e-06,True
7,MM3,1.000000e-07,True
8,MM3,1.000000e-08,True
